In [1]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle
import yaml
import pandas as pd

from lib.notebooks import plot_groups_metrics
from quantizer import Quantizer

In [2]:
quantizers_path = glob("../out/quantizer/*/")
quantizers_path.sort()

In [18]:
groups_metrics = {}
quantizers_loss = {
    "datasets": [],
    "hidden_layers": [],
    "dropout_p": [],
    "learning_rate": [],
    "commitment_cost": [],
    "total_loss": [],
    "reconstruction_error": [],
    "vq_loss": [],
}

for quantizer_path in quantizers_path:
    quantizer = Quantizer.reload(quantizer_path, load_nn=False)
    config = quantizer.config
    
    with open("%s/metrics.pickle" % quantizer_path, "rb") as f:
        metrics = pickle.load(f)
    final_loss_index = np.argmin(metrics["validation"]["total_loss"])
    final_total_loss = metrics["test"]["total_loss"][final_loss_index]
    final_reconstruction_error = metrics["test"]["reconstruction_error"][final_loss_index]
    final_vq_loss = metrics["test"]["vq_loss"][final_loss_index]
    
    quantizers_loss["datasets"].append(",".join(config['dataset']['names']))
    quantizers_loss["hidden_layers"].append(f"{len(config['model']['hidden_dims'])}x{config['model']['hidden_dims'][0]}")
    quantizers_loss["dropout_p"].append(config['model']['dropout_p'])
    quantizers_loss["learning_rate"].append(config['training']['learning_rate'])
    quantizers_loss["commitment_cost"].append(config['model']['commitment_cost'])
    
    quantizers_loss["total_loss"].append(final_total_loss)
    quantizers_loss["reconstruction_error"].append(final_reconstruction_error)
    quantizers_loss["vq_loss"].append(final_vq_loss)
    
    group_name = "\n".join((
        f"{','.join(config['dataset']['names'])}",
        f"hidden_layers={len(config['model']['hidden_dims'])}x{config['model']['hidden_dims'][0]}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][quantizer_path] = metrics

quantizers_loss = pd.DataFrame(quantizers_loss)

In [20]:
quantizers_loss.sort_values("reconstruction_error").head(10)

,datasets,hidden_layers,dropout_p,learning_rate,commitment_cost,total_loss,reconstruction_error,vq_loss
45,"pb2007,gb2016,th2016",1x512,0.214496,0.001188,1.482109,0.558082,0.381791,0.176291
33,"pb2007,gb2016,th2016",2x256,0.097725,0.000618,1.428386,0.465298,0.382452,0.082846
38,"pb2007,gb2016,th2016",1x128,0.016934,0.001394,1.861183,0.590471,0.385426,0.205045
96,"pb2007,gb2016,th2016",1x256,0.269028,0.001697,1.953235,0.531201,0.390863,0.140338
82,"pb2007,gb2016,th2016",1x512,0.485087,0.000482,1.560157,0.567754,0.391204,0.176550
9,"pb2007,gb2016,th2016",2x512,0.255175,0.000252,0.586623,0.510588,0.391880,0.118708
36,"pb2007,gb2016,th2016",3x256,0.117205,0.000233,0.786270,0.466884,0.393574,0.073309
47,"pb2007,gb2016,th2016",4x512,0.306015,0.000184,1.938586,0.417760,0.394200,0.023560
97,"pb2007,gb2016,th2016",4x256,0.257288,0.000307,1.736629,0.414983,0.394786,0.020196
100,"pb2007,gb2016,th2016",4x512,0.259563,0.000201,1.380088,0.428938,0.394932,0.034006


In [5]:
metrics_name = [
    "total_loss",
    "reconstruction_error",
    "vq_loss",
]

def show_metrics(split_name="validation"):
    plot_groups_metrics(groups_metrics, metrics_name, split_name)
display(ipw.interactive(show_metrics, split_name=["train", "validation"]))

interactive(children=(Dropdown(description='split_name', index=1, options=('train', 'validation'), value='vali…